import numpy as np
import quaternion

T=10
d=np.ones(3)
F_B=d*T
print(F_B.shape)
q=np.quaternion(0,4/5,0,3/5)

def vec2quat(vec):
    return np.quaternion(0,vec[0],vec[1],vec[2])
def quat23dvec(quat):
    return np.array([quat.x,quat.y,quat.z])
def cross_operator(x): #assuming x is shape (1,dim)
    ret=np.zeros((x.shape[0],x.shape[0]))
    for t in range(x.shape[0]):
        ret[t][t]=x[t]
    return ret
def quat24dvec(x):
    return np.array([x.w,x.x,x.y,x.z])
def omegaOperator(x):
    return np.array([
                     [0,-x[0],-x[1],-x[2]],
                     [x[0],0,x[2],-x[1]],
                     [x[1],-x[2],0,x[0]],
                     [x[2],x[1],-x[0],0]])

F_N=quat23dvec(q*vec2quat(F_B)*q.conjugate())
r=np.ones(3)
L_B=np.cross(r,F_B)
L_B

cross_operator(np.array([1,2,3]))

J=np.array([[2,0,0],[0,3,0],[0,0,4]])

J_inv=np.linalg.inv(J)
w=np.array([1,2,4])
w_dot=-J_inv@cross_operator(w)@J@w+J_inv@L_B

q_dot=vec2quat(w)*q/2
print(q_dot)
q_dot=quat24dvec(q_dot)

q_dot

In [1]:
import numpy as np
import sys
from casadi import *


# Import do_mpc package:
import do_mpc
T=10
r=np.array([0,0,-1])

In [2]:
model_type = 'discrete' # either 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)

In [3]:
r=model.set_variable('_x','linear',(3))
v=model.set_variable('_x','dlinear',(3))
m=model.set_variable('_x','mass',(1))
q=model.set_variable('_x','attitude',(4))
omega=model.set_variable('_x','omega',(3))
g=np.array([0,0,-9.81])
d=model.set_variable('_u','tvc',(3))
# L_B=model.set_variable('_z','torque',(3))
# F_N=model.set_variable('_z','force',(3))
# J=model.set_variable('_z','Inertia',(3,3))
# omega_crap=model.set_variable('_z','omegacrap',(3,3))
# omega_ham=model.set_variable('_z','omegaham',(4,4))

In [4]:
F_B=d*T

In [5]:
F_N=SX.sym('force_in',3)

In [6]:
x,y,z=vertsplit(F_B)
qw,qx,qy,qz=vertsplit(q)
print(qw)
F_N[0] = x*(qx*qx+qw*qw-qy*qy- qz*qz) + y*(2*qx*qy- 2*qw*qz) + z*(2*qx*qz+ 2*qw*qy)
F_N[1] = x*(2*qw*qz + 2*qx*qy) + y*(qw*qw - qx*qx+ qy*qy - qz*qz)+ z*(-2*qw*qx+ 2*qy*qz)
F_N[2] = x*(-2*qw*qy+ 2*qx*qz) + y*(2*qw*qx+ 2*qy*qz)+ z*(qw*qw - qx*qx- qy*qy+ qz*qz)

attitude_0


In [7]:
J=SX.sym('moi',3,3)
J[0,0]=10
J[1,1]=10
J[2,2]=10

In [8]:
op=SX.sym('op',3,3)
op[0,0]=0
op[0,1]=-omega[2]
op[0,2]=omega[1]
op[1,0]=omega[2]
op[1,1]=0
op[1,2]=-omega[0]
op[2,0]=-omega[1]
op[2,1]=omega[0]
op[2,2]=0

In [9]:
L_B=cross(r,F_B)

In [10]:
oh=SX.sym('oh',4,4)
oh[0,0]=0
oh[0,1]=-omega[0]
oh[0,2]=-omega[1]
oh[0,3]=-omega[2]
oh[1,0]=omega[0]
oh[1,1]=0
oh[1,2]=omega[2]
oh[1,3]=-omega[1]
oh[2,0]=omega[1]
oh[2,1]=-omega[2]
oh[2,2]=0
oh[2,3]=omega[0]
oh[3,0]=omega[2]
oh[3,1]=omega[1]
oh[3,2]=-omega[0]
oh[3,3]=0
oh

SX(@1=0, 
[[@1, (-omega_0), (-omega_1), (-omega_2)], 
 [omega_0, @1, omega_2, (-omega_1)], 
 [omega_1, (-omega_2), @1, omega_0], 
 [omega_2, omega_1, (-omega_0), @1]])

In [11]:
J_inv=inv(J)
domega=-J_inv@op@J@omega+L_B
dq=(oh@q)/2
print(domega.shape)

(3, 1)


In [12]:
dv=(F_N/m)+g

In [13]:
dm=(sqrt(F_B[0]**2+F_B[1]**2+F_B[2]**2))/100 # 100 is I_sp*g_ref

In [14]:
model.set_rhs('attitude',dq)
model.set_rhs('omega',domega)
model.set_rhs('dlinear',dv)
model.set_rhs('linear',v)
model.set_rhs('mass',dm)

In [15]:
model.rhs_list

[{'var_name': 'attitude',
  'expr': SX(@1=2, [(-((((omega_0*attitude_1)+(omega_1*attitude_2))+(omega_2*attitude_3))/@1)), ((((omega_0*attitude_0)+(omega_2*attitude_2))-(omega_1*attitude_3))/@1), ((((omega_1*attitude_0)-(omega_2*attitude_1))+(omega_0*attitude_3))/@1), ((((omega_2*attitude_0)+(omega_1*attitude_1))-(omega_0*attitude_2))/@1)])},
 {'var_name': 'omega',
  'expr': SX(@1=10, @2=100, @3=(((@1*(@2-(moi_7*moi_5)))-(moi_3*((@1*moi_1)-(moi_7*moi_2))))+(moi_6*((moi_1*moi_5)-(@1*moi_2)))), @4=(((@1*moi_3)-(moi_6*moi_5))/@3), @5=(((moi_3*moi_7)-(@1*moi_6))/@3), @6=((@4*omega_2)+(@5*omega_1)), @7=((@2-(moi_7*moi_5))/@3), @8=((@7*omega_2)-(@5*omega_0)), @9=((@7*omega_1)+(@4*omega_0)), @10=(@1*tvc_2), @11=(@1*tvc_1), @12=(((@1*moi_7)-(moi_6*moi_1))/@3), @13=(((@1*moi_1)-(moi_7*moi_2))/@3), @14=((@12*omega_0)-(@13*omega_2)), @15=((@2-(moi_6*moi_2))/@3), @16=((@15*omega_2)+(@12*omega_1)), @17=((@13*omega_1)+(@15*omega_0)), @18=(@1*tvc_0), @19=(((@1*moi_5)-(moi_3*moi_2))/@3), @20=((@2-(moi_

In [16]:
model.setup()